###### Scrapyard notebook overview

In this notebook I store select jetissoned paths as well as previous verions of crucial functions in case a new version turns out to be broken.  I intend to delete this notebook prior to making the repo public.

[player class source](https://towardsdatascience.com/rating-sports-teams-elo-vs-win-loss-d46ee57c1314)

In [ ]:

class Player(object):
    """Player Object"""
    def __init__(self, name, rtg, elo, nudge):
        super(Player, self).__init__()
        self.name = name
        # true rating
        self.rtg = rtg
        # elo rating
        self.elo = elo
        # wins, losses, ties
        self.wins = 0
        self.losses = 0
        self.ties = 0
        self.games_played = 0
        # how much to nudge ratings
        self.nudge = nudge
    def nudge_rating(self):
        # decide to nudge up or down
        direction = random.randint(0,1)
        if direction == 0:
            self.rtg += self.nudge
        else:
            self.rtg -= self.nudge
    def add_win(self):
        self.wins += 1
    def add_loss(self):
        self.losses += 1
    def add_tie(self):
        self.ties += 1
    def played_game(self):
        self.games_played += 1

In [4]:
# idea: have an id, an elo rating, a glicko rating, etc. all
# in one player class
# class Competitor:
#     def __init__(self,player_id,elo_rtg):
#         self.elo_

IndentationError: expected an indented block (4289044415.py, line 5)

In [ ]:
# this version of the epoch function has the player id in 
# the format of a string with p_ prepended to it

def epoch(matches, players_list, players_dict):
    '''Take in a dataframe with matches, a list of players, a dictionary of players with the elements
    of player list as keys and an instance of the glicko2 class Player() as the value.
    This function wll return the updated players_list and players_dict.
    
    Future iterations of this function should do away with the p_ , and only should need the 
    players_dict with the keys as integers indicating the player id.
    '''
    
    players = ['p_' + str(p) for p in np.append(matches['winner_id'].unique(),\
                                                matches['loser_id'].unique())]
    # instantiate player class for players not yet instantiated
    players_to_instantiate = list(set(players) - set(players_list))
    new_players = {player: Player() for player in players_to_instantiate}
    # print(type(new_players))

    # update list of instantiated players
    players_list = list(set(players).union(set(players_list)))
    # update dictionary of instantiated players
    players_dict.update(new_players)
    # print(type(players_dict))
    # determine who competed and who didn't
    players_dnc = list(set(players_list) - set(players))
    players_compete = list(set(players_list)-set(players_dnc))
    # print(f'{players_compete=}')
    # update attributes for players who didn't compete
    for player in players_dnc:
        players_dict[player].did_not_compete()
    # fill dictionary with (wins,losses), a tuple of list of opponents in wins and losses for each match
    results = {}
    for player in players_compete:
        wins = ['p_' + str(winner) for winner in matches[matches['winner_id']==int(player[2:])]['loser_id']]
        losses = ['p_' + str(winner) for winner in matches[matches['loser_id']==int(player[2:])]['winner_id']]
        # get opponents' rating and rd
        # print(f'{wins=}')
        losses_rating = [players_dict[loss].getRating() for loss in losses ]
        losses_rd = [players_dict[loss].getRd() for loss in losses ]
        wins_rating = [players_dict[win].getRating() for win in wins ]
        wins_rd = [players_dict[win].getRd() for win in wins ]
        outcomes = np.append(np.ones(len(wins)),np.zeros(len(losses)))
        opponent_rating = wins_rating + losses_rating
        # print(f'{opponent_rating=}')
        opponent_rd = wins_rd + losses_rd
        results[player] = (opponent_rating, opponent_rd, outcomes)
        
    # update players
    for player in list(results.keys()):
        (rating_list, RD_list, outcome_list) = results[player]
        # print(f'{rating_list=}, {RD_list=}, {outcome_list=}')
        players_dict[player].update_player(rating_list, RD_list, outcome_list)

    return (players_list,players_dict)
    # opponent_indices = matches[matches['loser_id'].str==player '']

In [ ]:
  
def epoch(matches, players_dict):
    '''Take in a dataframe with matches, a list of players, a dictionary of players with the elements
    of player list as keys and an instance of the glicko2 class Player() as the value.
    This function wll return the updated players_list and players_dict.
    
    Future iterations of this function should do away with the p_ , and only should need the 
    players_dict with the keys as integers indicating the player id.
    '''
    players_list = list(players_dict.keys())
    players = [p for p in np.append(matches['winner_id'].unique(),\
                                                matches['loser_id'].unique())]
    # instantiate player class for players not yet instantiated
    players_to_instantiate = list(set(players) - set(players_list))
    
    new_players = {player: Player() for player in players_to_instantiate}
    # print(type(new_players))

    # update list of instantiated players
    players_list = list(set(players).union(set(players_list)))
    # update dictionary of instantiated players
    players_dict.update(new_players)
    # print(type(players_dict))
    # determine who competed and who didn't
    players_dnc = list(set(players_list) - set(players))
    players_compete = list(set(players_list)-set(players_dnc))
    # print(f'{players_compete=}')
    # update attributes for players who didn't compete
    for player in players_dnc:
        players_dict[player].did_not_compete()
    # fill dictionary with (wins,losses), a tuple of list of opponents in wins and losses for each match
    results = {}
    for player in players_compete:
        wins = [winner for winner in matches[matches['winner_id']==player]['loser_id']]
        losses = [loser for loser in matches[matches['loser_id']== player]['winner_id']]
        # get opponents' rating and rd
        # print(f'{wins=}')
        losses_rating = [players_dict[loss].getRating() for loss in losses ]
        losses_rd = [players_dict[loss].getRd() for loss in losses ]
        wins_rating = [players_dict[win].getRating() for win in wins ]
        wins_rd = [players_dict[win].getRd() for win in wins ]
        outcomes = np.append(np.ones(len(wins)),np.zeros(len(losses)))
        opponent_rating = wins_rating + losses_rating
        # print(f'{opponent_rating=}')
        opponent_rd = wins_rd + losses_rd
        results[player] = (opponent_rating, opponent_rd, outcomes)
        
    # update players
    for player in list(results.keys()):
        (rating_list, RD_list, outcome_list) = results[player]
        # print(f'{rating_list=}, {RD_list=}, {outcome_list=}')
        players_dict[player].update_player(rating_list, RD_list, outcome_list)

    return players_dict
    # opponent_indices = matches[matches['loser_id'].str==player '']


def epoch_3(matches, players_dict):
    '''Take in a dataframe with matches, a list of players, a dictionary of players with the elements
    of player list as keys and an instance of the glicko2 class Player() as the value.
    This function wll return the updated players_list and players_dict.
    
    Future iterations of this function should do away with the p_ , and only should need the 
    players_dict with the keys as integers indicating the player id.
    '''
    players_list = list(players_dict.keys())
    players = [p for p in np.append(matches['winner_id'].unique(),\
                                                matches['loser_id'].unique())]
    # instantiate player class for players not yet instantiated
    players_to_instantiate = list(set(players) - set(players_list))
    
    new_players = {player: Player() for player in players_to_instantiate}
    # print(type(new_players))

    # update list of instantiated players
    players_list = list(set(players).union(set(players_list)))
    # update dictionary of instantiated players
    players_dict.update(new_players)
    # print(type(players_dict))
    # determine who competed and who didn't
    players_dnc = list(set(players_list) - set(players))
    players_compete = list(set(players_list)-set(players_dnc))
    # print(f'{players_compete=}')
    # update attributes for players who didn't compete
    for player in players_dnc:
        players_dict[player].did_not_compete()
    # fill dictionary with (wins,losses), a tuple of list of opponents in wins and losses for each match
    results = {}
    for player in players_compete:
        wins = [winner for winner in matches[matches['winner_id']==player]['loser_id']]
        losses = [loser for loser in matches[matches['loser_id']== player]['winner_id']]
        # get opponents' rating and rd
        # print(f'{wins=}')
        losses_rating = [players_dict[loss].getRating() for loss in losses ]
        losses_rd = [players_dict[loss].getRd() for loss in losses ]
        wins_rating = [players_dict[win].getRating() for win in wins ]
        wins_rd = [players_dict[win].getRd() for win in wins ]
        outcomes = np.append(np.ones(len(wins)),np.zeros(len(losses)))
        opponent_rating = wins_rating + losses_rating
        # print(f'{opponent_rating=}')
        opponent_rd = wins_rd + losses_rd
        results[player] = (opponent_rating, opponent_rd, outcomes)
        
    # update players
    for player in list(results.keys()):
        (rating_list, RD_list, outcome_list) = results[player]
        # print(f'{rating_list=}, {RD_list=}, {outcome_list=}')
        players_dict[player].update_player(rating_list, RD_list, outcome_list)

    return players_dict
    # opponent_indices = matches[matches['loser_id'].str==player '']
    
def epoch_2(matches, players_list, players_dict):
    '''Take in a dataframe with matches, a list of players, a dictionary of players with the elements
    of player list as keys and an instance of the glicko2 class Player() as the value.
    This function wll return the updated players_list and players_dict.
    
    Future iterations of this function should do away with the p_ , and only should need the 
    players_dict with the keys as integers indicating the player id.
    '''
    
    players = [p for p in np.append(matches['winner_id'].unique(),\
                                                matches['loser_id'].unique())]
    # instantiate player class for players not yet instantiated
    players_to_instantiate = list(set(players) - set(players_list))
    new_players = {player: Player() for player in players_to_instantiate}
    # print(type(new_players))

    # update list of instantiated players
    players_list = list(set(players).union(set(players_list)))
    # update dictionary of instantiated players
    players_dict.update(new_players)
    # print(type(players_dict))
    # determine who competed and who didn't
    players_dnc = list(set(players_list) - set(players))
    players_compete = list(set(players_list)-set(players_dnc))
    # print(f'{players_compete=}')
    # update attributes for players who didn't compete
    for player in players_dnc:
        players_dict[player].did_not_compete()
    # fill dictionary with (wins,losses), a tuple of list of opponents in wins and losses for each match
    results = {}
    for player in players_compete:
        wins = [winner for winner in matches[matches['winner_id']==player]['loser_id']]
        losses = [loser for loser in matches[matches['loser_id']== player]['winner_id']]
        # get opponents' rating and rd
        # print(f'{wins=}')
        losses_rating = [players_dict[loss].getRating() for loss in losses ]
        losses_rd = [players_dict[loss].getRd() for loss in losses ]
        wins_rating = [players_dict[win].getRating() for win in wins ]
        wins_rd = [players_dict[win].getRd() for win in wins ]
        outcomes = np.append(np.ones(len(wins)),np.zeros(len(losses)))
        opponent_rating = wins_rating + losses_rating
        # print(f'{opponent_rating=}')
        opponent_rd = wins_rd + losses_rd
        results[player] = (opponent_rating, opponent_rd, outcomes)
        
    # update players
    for player in list(results.keys()):
        (rating_list, RD_list, outcome_list) = results[player]
        # print(f'{rating_list=}, {RD_list=}, {outcome_list=}')
        players_dict[player].update_player(rating_list, RD_list, outcome_list)

    return (players_list,players_dict)
    # opponent_indices = matches[matches['loser_id'].str==player '']
    
def cutDf(df,start,end,column):
    
    return df[(df[column]>=df[column][df[column].index[start]])&
              (df[column]<=df[column][df[column].index[end]])]

def epochs(match_history,interval_length = 365):
    '''Calculate the ending rating for each player with the lengh of each epoch being
    a funtion of the interval_length'''
    
    max_date = max(match_history['tourney_date']) # maximum date in the records
    min_date = min(match_history['tourney_date']) # minimum date in the records
    date_range = range(0,(max_date-min_date).days + 1,interval_length) # days from date of first
    # match in increments of the interval (default 365).  This is the length of each epoch. 
    players_list = []
    players_dict = {}
    
    epoch_cutoffs = [min_date + timedelta(days = x) for x in date_range]
    print(epoch_cutoffs)
    epoch_ranges = zip(epoch_cutoffs[0:-2],epoch_cutoffs[1:-1])
    
    
    # iteratively re-update for each epoch
    for period in epoch_ranges:
        rating_period = match_history[(match_history['tourney_date']>=period[0])&(match_history['tourney_date']<period[1])]
        if not rating_period.empty:
            (players_list,players_dict) = epoch(rating_period,players_list,players_dict)
        else:
            for player in players_list:
                players_dict[player].did_not_compete()
    # get the final, extended rating period updates.
    rating_period = match_history[match_history['tourney_date']>=epoch_cutoffs[-1]]
    (players_list,players_dict) = epoch(rating_period,players_list,players_dict)
    return (players_list,players_dict)

In [ ]:
def epochElo2(matches, players_dict, cutoff_date):
    '''Take in a dataframe with matches, a list of players, a dictionary of players 
    with the elements
    of player list as keys and an instance of the glicko2 class Player() as the value.
    This function wll return the updated players_list and players_dict.
    
    Future iterations of this function should do away with the p_ , and only should need the 
    players_dict with the keys as integers indicating the player id.
    '''
    players_list = list(players_dict.keys())
    players = [p for p in np.append(matches['winner_id'].unique(),\
                                                matches['loser_id'].unique())]
    # instantiate player class for players not yet instantiated
    players_to_instantiate = list(set(players) - set(players_list))
    
    new_players = {player: PlayerElo() for player in players_to_instantiate}
    # print(type(new_players))

    # update list of instantiated players
    players_list = list(set(players).union(set(players_list)))
    # update dictionary of instantiated players
    players_dict.update(new_players)
    # print(type(players_dict))
    # determine who competed and who didn't
    players_dnc = list(set(players_list) - set(players))
    players_compete = list(set(players_list)-set(players_dnc))
    # fill dictionary with player:(wins,losses), a tuple of list of opponents in wins and losses for each match
    results = {}
    for player in players_compete:
        wins = [winner for winner in matches[matches['winner_id']==player]['loser_id']]
        losses = [loser for loser in matches[matches['loser_id']== player]['winner_id']]
        losses_rating = np.array([players_dict[loss].getRating() for loss in losses])
        wins_rating = np.array([players_dict[win].getRating() for win in wins])
        outcomes = np.append(np.ones(len(wins)),np.zeros(len(losses)))
        opponent_rating = np.append(wins_rating,losses_rating)
        # print(f'{opponent_rating=}')
        results[player] = (opponent_rating, outcomes)
    ratings_timestamp = {}
    # update players - this has to be done after results are gathered for all players.
    for player in list(results.keys()):
        (rating_list, outcome_list) = results[player]
        # print(f'{rating_list=}, {RD_list=}, {outcome_list=}')
        players_dict[player].update_player(rating_list,outcome_list)
        ratings_timestamp[(player,cutoff_date)] = players_dict[player].getRating()
    return players_dict,ratings_timestamp
    # opponent_indices = matches[matches['loser_id'].str==player '']

In [102]:
def epochs(match_history,interval_length = 365):
    '''Calculate the ending rating for each player with the lengh of each epoch being
    a funtion of the interval_length'''
    
    max_date = max(match_history['tourney_date']) # maximum date in the records
    min_date = min(match_history['tourney_date']) # minimum date in the records
    date_range = range(0,(max_date-min_date).days + 1,interval_length) # days from date of first
    # match in increments of the interval (default 365).  This is the length of each epoch. 
    players_list = []
    players_dict = {}
    
    epoch_cutoffs = [min_date + timedelta(days = x) for x in date_range]
    print(epoch_cutoffs)
    epoch_ranges = zip(epoch_cutoffs[0:-2],epoch_cutoffs[1:-1])
    
    
    # iteratively re-update for each epoch
    for period in epoch_ranges:
        rating_period = match_history[(match_history['tourney_date']>=period[0])&(match_history['tourney_date']<period[1])]
        if not rating_period.empty:
            (players_list,players_dict) = epoch(rating_period,players_list,players_dict)
        else:
            for player in players_list:
                players_dict[player].did_not_compete()
    # get the final, extended rating period updates.
    rating_period = match_history[match_history['tourney_date']>=epoch_cutoffs[-1]]
    (players_list,players_dict) = epoch(rating_period,players_list,players_dict)
    return (players_list,players_dict)
    

In [5]:
def epochElo(matches, players_dict, cutoff_date):
    '''Take in a dataframe with matches, a list of players, a dictionary of players 
    with the elements
    of player list as keys and an instance of the glicko2 class Player() as the value.
    This function wll return the updated players_list and players_dict.
    
    Future iterations of this function should do away with the p_ , and only should need the 
    players_dict with the keys as integers indicating the player id.
    '''
    players_list = list(players_dict.keys())
    players = [p for p in np.append(matches['winner_id'].unique(),\
                                                matches['loser_id'].unique())]
    # instantiate player class for players not yet instantiated
    players_to_instantiate = list(set(players) - set(players_list))
    
    new_players = {player: PlayerElo() for player in players_to_instantiate}
    # print(type(new_players))

    # update list of instantiated players
    players_list = list(set(players).union(set(players_list)))
    # update dictionary of instantiated players
    players_dict.update(new_players)
    # print(type(players_dict))
    # determine who competed and who didn't
    players_dnc = list(set(players_list) - set(players))
    players_compete = list(set(players_list)-set(players_dnc))
    # fill dictionary with player:(wins,losses), a tuple of list of opponents in wins and losses for each match
    results = {}
    for player in players_compete:
        wins = [winner for winner in matches[matches['winner_id']==player]['loser_id']]
        losses = [loser for loser in matches[matches['loser_id']== player]['winner_id']]
        losses_rating = [players_dict[loss].getRating() for loss in losses]
        wins_rating = [players_dict[win].getRating() for win in wins]
        outcomes = np.append(np.ones(len(wins)),np.zeros(len(losses)))
        opponent_rating = wins_rating + losses_rating
        # print(f'{opponent_rating=}')
        results[player] = (opponent_rating, outcomes)
        
    ratings_timestamp = {}
    # update players
    for player in list(results.keys()):
        (rating_list, outcome_list) = results[player]
        # print(f'{rating_list=}, {RD_list=}, {outcome_list=}')
        players_dict[player].update_player(rating_list,outcome_list)
        ratings_timestamp[(player,cutoff_date)] = players_dict[player].getRating()
    return players_dict,ratings_timestamp
    # opponent_indices = matches[matches['loser_id'].str==player '']

In [82]:
# function to get all unique timestamps:
def getUniqueTimestamps(ratingHistory):
    '''take in rating histories and return the unique timestamps.'''
    times = set()
    for key in ratingHistory.keys():
        times.add(key[1])
    return times

In [1]:
def epochG(matches, players_dict,cutoff_date):
    '''Take in a dataframe with matches, a list of players, a dictionary of players with the elements
    of player list as keys and an instance of the glicko2 class Player() as the value.
    This function wll return the updated players_list and players_dict.'''
    
    players_list = list(players_dict.keys())
    players = [p for p in np.append(matches['winner_id'].unique(),\
                                                matches['loser_id'].unique())]
    # instantiate player class for players not yet instantiated
    players_to_instantiate = list(set(players) - set(players_list))
    
    new_players = {player: Player() for player in players_to_instantiate}
    # print(type(new_players))
    players_list = list(set(players).union(set(players_list)))

    # update list of instantiated players
    # update dictionary of instantiated players
    players_dict.update(new_players)
    # print(type(players_dict))
    # determine who competed and who didn't
    players_dnc = list(set(players_list) - set(players))
    players_compete = players
    # print(f'{players_compete=}')
    # update rating deviation for players who didn't compete
    for player in players_dnc:
        players_dict[player].did_not_compete()
    # fill dictionary with (wins,losses), a tuple of list of opponents in wins and losses for each match
    results = {}
    # update players who did compete
    for player in players_compete:
        # get id of players they beat
        wins = [winner for winner in matches[matches['winner_id']==player]['loser_id']]
        # get id of players they lost to
        losses = [loser for loser in matches[matches['loser_id']== player]['winner_id']]
        # get opponents' rating and rd
        losses_rating = [players_dict[loss].getRating() for loss in losses]
        losses_rd = [players_dict[loss].getRd() for loss in losses]
        wins_rating = [players_dict[win].getRating() for win in wins]
        wins_rd = [players_dict[win].getRd() for win in wins]
        outcomes = np.append(np.ones(len(wins)),np.zeros(len(losses)))
        opponent_rating = wins_rating + losses_rating
        # print(f'{opponent_rating=}')
        opponent_rd = wins_rd + losses_rd
        results[player] = (opponent_rating, opponent_rd, outcomes)
    
    ratings_timestamp = {}
    # update players
    for player in list(results.keys()):
        (rating_list, RD_list, outcome_list) = results[player]
        players_dict[player].update_player(rating_list, RD_list, outcome_list)
        ratings_timestamp[(player,cutoff_date)] = players_dict[player].getRating()
        
    return players_dict,ratings_timestamp


In [70]:
def epochsG(match_history, interval_length = 365):
    '''Calculate the ending rating for each player with the lengh of each epoch being
    a funtion of the interval_length
    
    Next iterations: generate a rating history indicating the rating of each player each time that
    the rating updates.
    '''
    # check if the match_history is empty, as if it is the function call will not complete.
    if match_history.empty:
        return "Try again with a non-empty match history!"
    max_date = max(match_history['tourney_date']) # maximum date in the records
    min_date = min(match_history['tourney_date']) # minimum date in the records
    date_range = range(0,(max_date-min_date).days + 1,interval_length) # days from date of first
    # match in increments of the interval (default 365).  This is the length of each epoch. 
    epoch_cutoffs = [min_date + timedelta(days = x) for x in date_range] # The times that
    # divide the matches into each epoch.
    epoch_ranges = zip(epoch_cutoffs[0:-2],epoch_cutoffs[1:-1]) # each epoch will include matches
    # greater than or equal to the first element, less than the second element for the zip
    # generator's respective item.
    # print([r for r in epoch_ranges])
    players_dict = {} # instantiate the dictionary that will hold a Player() class for each player.
    ratings_history = {}

    # iteratively re-update for each epoch
    for period in epoch_ranges:
        # rating_ period is the df of matches that fall within a given epoch
        rating_period = match_history[(match_history['tourney_date']>=period[0])&
                                      (match_history['tourney_date']<period[1])]
        # If the rating period is empty, then adjust the rating deviation of the players.
        if not rating_period.empty:
            players_dict,ratings_timestamp = epochG(rating_period,players_dict,period[1])
            ratings_history.update(ratings_timestamp)
        else:
            for player in players_dict:
                players_dict[player].did_not_compete()
    # get the final rating period updates (for matches in the final 365 to 729 days).
    rating_period = match_history[match_history['tourney_date']>=epoch_cutoffs[-1]]
    
    players_dict,ratings_timestamp = epochG(rating_period,players_dict,max_date)
    ratings_history.update(ratings_timestamp)
    return players_dict, ratings_history

In [ ]:
def epochs2(match_history,interval_length = 365):
    '''Calculate the ending rating for each player with the lengh of each epoch being
    a funtion of the interval_length'''
    
    max_date = max(match_history['tourney_date']) # maximum date in the records
    min_date = min(match_history['tourney_date']) # minimum date in the records
    date_range = range(0,(max_date-min_date).days + 1,interval_length) # days from date of first
    # match in increments of the interval (default 365).  This is the length of each epoch. 
    players_list = []
    players_dict = {}
    
    epoch_cutoffs = [min_date + timedelta(days = x) for x in date_range]
    print(epoch_cutoffs)
    epoch_ranges = zip(epoch_cutoffs[0:-2],epoch_cutoffs[1:-1])
    
    
    # iteratively re-update for each epoch
    for period in epoch_ranges:
        rating_period = match_history[(match_history['tourney_date']>=period[0])&(match_history['tourney_date']<period[1])]
        if not rating_period.empty:
            players_dict = epoch(rating_period,players_dict)
        else:
            for player in list(players_dict.keys()):
                players_dict[player].did_not_compete()
    # get the final, extended rating period updates.
    rating_period = match_history[match_history['tourney_date']>=epoch_cutoffs[-1]]
    (players_list,players_dict) = epoch(rating_period,players_list,players_dict)
    return (players_list,players_dict)

In [ ]:
def epochs(match_history,interval_length = 365):
    '''Calculate the ending rating for each player with the lengh of each epoch being
    a funtion of the interval_length'''
    
    max_date = max(match_history['tourney_date']) # maximum date in the records
    min_date = min(match_history['tourney_date']) # minimum date in the records
    date_range = range(0,(max_date-min_date).days + 1,interval_length) # days from date of first
    # match in increments of the interval (default 365).  This is the length of each epoch. 
    players_list = []
    players_dict = {}
    
    epoch_cutoffs = [min_date + timedelta(days = x) for x in date_range]
    print(epoch_cutoffs)
    epoch_ranges = zip(epoch_cutoffs[0:-2],epoch_cutoffs[1:-1])
    
    
    # iteratively re-update for each epoch
    for period in epoch_ranges:
        rating_period = match_history[(match_history['tourney_date']>=period[0])&(match_history['tourney_date']<period[1])]
        if not rating_period.empty:
            (players_list,players_dict) = epoch(rating_period,players_list,players_dict)
        else:
            for player in players_list:
                players_dict[player].did_not_compete()
    # get the final, extended rating period updates.
    rating_period = match_history[match_history['tourney_date']>=epoch_cutoffs[-1]]
    (players_list,players_dict) = epoch(rating_period,players_list,players_dict)
    return (players_list,players_dict)

In [2]:
def epoch2(matches, players_list, players_dict):
    '''Take in a dataframe with matches, a list of players, a dictionary of players with the elements
    of player list as keys and an instance of the glicko2 class Player() as the value.
    This function wll return the updated players_list and players_dict.
    
    Future iterations of this function should do away with the p_ , and only should need the 
    players_dict with the keys as integers indicating the player id.
    '''
    
    players = [p for p in np.append(matches['winner_id'].unique(),\
                                                matches['loser_id'].unique())]
    # instantiate player class for players not yet instantiated
    players_to_instantiate = list(set(players) - set(players_list))
    new_players = {player: Player() for player in players_to_instantiate}
    # print(type(new_players))

    # update list of instantiated players
    players_list = list(set(players).union(set(players_list)))
    # update dictionary of instantiated players
    players_dict.update(new_players)
    # print(type(players_dict))
    # determine who competed and who didn't
    players_dnc = list(set(players_list) - set(players))
    players_compete = list(set(players_list)-set(players_dnc))
    # print(f'{players_compete=}')
    # update attributes for players who didn't compete
    for player in players_dnc:
        players_dict[player].did_not_compete()
    # fill dictionary with (wins,losses), a tuple of list of opponents in wins and losses for each match
    results = {}
    for player in players_compete:
        wins = [winner for winner in matches[matches['winner_id']==player]['loser_id']]
        losses = [loser for loser in matches[matches['loser_id']== player]['winner_id']]
        # get opponents' rating and rd
        # print(f'{wins=}')
        losses_rating = [players_dict[loss].getRating() for loss in losses ]
        losses_rd = [players_dict[loss].getRd() for loss in losses ]
        wins_rating = [players_dict[win].getRating() for win in wins ]
        wins_rd = [players_dict[win].getRd() for win in wins ]
        outcomes = np.append(np.ones(len(wins)),np.zeros(len(losses)))
        opponent_rating = wins_rating + losses_rating
        # print(f'{opponent_rating=}')
        opponent_rd = wins_rd + losses_rd
        results[player] = (opponent_rating, opponent_rd, outcomes)
        
    # update players
    for player in list(results.keys()):
        (rating_list, RD_list, outcome_list) = results[player]
        # print(f'{rating_list=}, {RD_list=}, {outcome_list=}')
        players_dict[player].update_player(rating_list, RD_list, outcome_list)

    return (players_list,players_dict)
    # opponent_indices = matches[matches['loser_id'].str==player '']

In [1]:
def epoch3(matches, players_dict):
    '''Take in a dataframe with matches, a list of players, a dictionary of players with the elements
    of player list as keys and an instance of the glicko2 class Player() as the value.
    This function wll return the updated players_list and players_dict.
    
    Future iterations of this function should do away with the p_ , and only should need the 
    players_dict with the keys as integers indicating the player id.
    '''
    players_list = list(players_dict.keys())
    players = [p for p in np.append(matches['winner_id'].unique(),\
                                                matches['loser_id'].unique())]
    # instantiate player class for players not yet instantiated
    players_to_instantiate = list(set(players) - set(players_list))
    
    new_players = {player: Player() for player in players_to_instantiate}
    # print(type(new_players))

    # update list of instantiated players
    players_list = list(set(players).union(set(players_list)))
    # update dictionary of instantiated players
    players_dict.update(new_players)
    # print(type(players_dict))
    # determine who competed and who didn't
    players_dnc = list(set(players_list) - set(players))
    players_compete = list(set(players_list)-set(players_dnc))
    # print(f'{players_compete=}')
    # update attributes for players who didn't compete
    for player in players_dnc:
        players_dict[player].did_not_compete()
    # fill dictionary with (wins,losses), a tuple of list of opponents in wins and losses for each match
    results = {}
    for player in players_compete:
        wins = [winner for winner in matches[matches['winner_id']==player]['loser_id']]
        losses = [loser for loser in matches[matches['loser_id']== player]['winner_id']]
        # get opponents' rating and rd
        # print(f'{wins=}')
        losses_rating = [players_dict[loss].getRating() for loss in losses ]
        losses_rd = [players_dict[loss].getRd() for loss in losses ]
        wins_rating = [players_dict[win].getRating() for win in wins ]
        wins_rd = [players_dict[win].getRd() for win in wins ]
        outcomes = np.append(np.ones(len(wins)),np.zeros(len(losses)))
        opponent_rating = wins_rating + losses_rating
        # print(f'{opponent_rating=}')
        opponent_rd = wins_rd + losses_rd
        results[player] = (opponent_rating, opponent_rd, outcomes)
        
    # update players
    for player in list(results.keys()):
        (rating_list, RD_list, outcome_list) = results[player]
        # print(f'{rating_list=}, {RD_list=}, {outcome_list=}')
        players_dict[player].update_player(rating_list, RD_list, outcome_list)

    return players_dict
    # opponent_indices = matches[matches['loser_id'].str==player '']

In [1]:
# epoch is the most current version of the function and will be called
# in the main notebook.

def epoch(matches, players_dict):
    '''Take in a dataframe with matches and a dictionary of players with the elements
    of player list as keys and an instance of the glicko2 class Player() as the value.
    This function wll return the updated players_dict.'''
    
    players_list = list(players_dict.keys())
    players = [p for p in np.append(matches['winner_id'].unique(),\
                                                matches['loser_id'].unique())]
    # instantiate player class for players not yet instantiated
    players_to_instantiate = list(set(players) - set(players_list))
    
    new_players = {player: Player() for player in players_to_instantiate}
    # print(type(new_players))

    # update list of instantiated players
    players_list = list(set(players).union(set(players_list)))
    # update dictionary of instantiated players
    players_dict.update(new_players)
    # print(type(players_dict))
    # determine who competed and who didn't
    players_dnc = list(set(players_list) - set(players))
    players_compete = list(set(players_list)-set(players_dnc))
    # print(f'{players_compete=}')
    # update attributes for players who didn't compete
    for player in players_dnc:
        players_dict[player].did_not_compete()
    # fill dictionary with (wins,losses), a tuple of list of opponents in wins and losses for each match
    results = {}
    for player in players_compete:
        wins = [winner for winner in matches[matches['winner_id']==player]['loser_id']]
        losses = [loser for loser in matches[matches['loser_id']== player]['winner_id']]
        # get opponents' rating and rd
        # print(f'{wins=}')
        losses_rating = [players_dict[loss].getRating() for loss in losses ]
        losses_rd = [players_dict[loss].getRd() for loss in losses ]
        wins_rating = [players_dict[win].getRating() for win in wins ]
        wins_rd = [players_dict[win].getRd() for win in wins ]
        outcomes = np.append(np.ones(len(wins)),np.zeros(len(losses)))
        opponent_rating = wins_rating + losses_rating
        # print(f'{opponent_rating=}')
        opponent_rd = wins_rd + losses_rd
        results[player] = (opponent_rating, opponent_rd, outcomes)
        
    # update players
    for player in list(results.keys()):
        (rating_list, RD_list, outcome_list) = results[player]
        # print(f'{rating_list=}, {RD_list=}, {outcome_list=}')
        players_dict[player].update_player(rating_list, RD_list, outcome_list)

    return players_dict
    # opponent_indices = matches[matches['loser_id'].str==player '']


In [6]:
# an implementation of a class that holds a player's rating,
# updates the rating, instantiate's the player.
# It ammounts to a simplified modeled of Ryan Kirkman's 
# glicko2 implementation.
import numpy as np # numpy will be used for vectorized calculations.
class PlayerElo:
    # initialize the update rate
    _k = 16 
    def getRating(self):
        return self.__rating
    def setRating(self, rating):
        self.__rating = rating
    def __init__(self, rating = 1500):
        self.__rating = rating
    def update_player(self, rating_list, outcome_list):
        _k = self._k
        rating_list = np.array(rating_list).astype(float) # avoid exponentiation of int 
        # by negative value error.
        n = len(rating_list)
        # calculate expected wins
        expected_wins = np.divide(1,(1 + np.power(10,(rating_list-self.__rating)/400)))
        self.__rating = self.__rating + _k*np.sum(outcome_list - expected_wins)

In [ ]:
def epochsElo(match_history, interval_length = 365):
    '''Calculate the ending rating for each player with the lengh of each epoch being
    a funtion of the interval_length
    
    Next iterations: generate a rating history indicating the rating of each player each time that
    the rating updates.
    '''
    # check if the match_history is empty, as if it is the function call will not complete.
    if match_history.empty:
        return "Try again with a non-empty match history!"
    max_date = max(match_history['tourney_date']) # maximum date in the records
    min_date = min(match_history['tourney_date']) # minimum date in the records
    date_range = range(0,(max_date-min_date).days + 1,interval_length) # days from date of first
    # match in increments of the interval (default 365).  This is the length of each epoch. 
    epoch_cutoffs = [min_date + timedelta(days = x) for x in date_range] # The times that
    # divide the matches into each epoch.
    epoch_ranges = zip(epoch_cutoffs[0:-2],epoch_cutoffs[1:-1]) # each epoch will include matches
    # greater than or equal to the first element, less than the second element for the zip
    # generator's respective item.
    players_dict = {} # instantiate the dictionary that will hold a PlayerElo() class for each player.
    ratings_history = {}
    # iteratively re-update for each epoch
    for period in epoch_ranges:
        # rating_period is the df of matches that fall within an epoch
        rating_period = match_history[(match_history['tourney_date']>=period[0])&(match_history['tourney_date']<period[1])]
        # make sure the rating period isn't empty and then update the players.
        if not rating_period.empty:
            players_dict,ratings_timestamp = epochElo(rating_period,players_dict,period[1])
            ratings_history.update(ratings_timestamp)
    # get the final rating period updates (for matches in the final 365 to 729 days).
    rating_period = match_history[match_history['tourney_date']>=epoch_cutoffs[-1]]
    players_dict, ratings_timestamp = epochElo(rating_period,players_dict,epoch_cutoffs[-1])
    ratings_history.update(ratings_timestamp)
    return players_dict,ratings_history